In [13]:
%load_ext autoreload
%autoreload 2
# Add custom functions to path
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import functions
import pandas as pd
from sqlalchemy import create_engine

# Credentials
from src import local
USER = local.user 
PASS = local.password
HOST = local.host
PORT = local.port

#create engine
engine = create_engine(f'postgresql://{USER}:{PASS}@{HOST}:{PORT}/match_finder')

query = """
SELECT rounds.*, date, time, 
final_round, timeformat
FROM rounds
JOIN bouts ON bouts.id = rounds.bout_id
JOIN events ON events.id = bouts.event_id
"""

rounds = pd.read_sql(query, engine)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
rounds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26214 entries, 0 to 26213
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fighter      26214 non-null  object
 1   kd           26214 non-null  int64 
 2   sba          26214 non-null  int64 
 3   ps           26214 non-null  int64 
 4   rev          26214 non-null  int64 
 5   ss_s         26214 non-null  int64 
 6   ss_a         26214 non-null  int64 
 7   h_ss_s       26214 non-null  int64 
 8   h_ss_a       26214 non-null  int64 
 9   b_ss_s       26214 non-null  int64 
 10  b_ss_a       26214 non-null  int64 
 11  l_ss_s       26214 non-null  int64 
 12  l_ss_a       26214 non-null  int64 
 13  d_ss_s       26214 non-null  int64 
 14  d_ss_a       26214 non-null  int64 
 15  c_ss_s       26214 non-null  int64 
 16  c_ss_a       26214 non-null  int64 
 17  g_ss_s       26214 non-null  int64 
 18  g_ss_a       26214 non-null  int64 
 19  ts_s         26214 non-nu

In [15]:
rounds.columns

Index(['fighter', 'kd', 'sba', 'ps', 'rev', 'ss_s', 'ss_a', 'h_ss_s', 'h_ss_a',
       'b_ss_s', 'b_ss_a', 'l_ss_s', 'l_ss_a', 'd_ss_s', 'd_ss_a', 'c_ss_s',
       'c_ss_a', 'g_ss_s', 'g_ss_a', 'ts_s', 'ts_a', 'td_s', 'td_a', 'outcome',
       'bout_id', 'fighter_id', 'round', 'date', 'time', 'final_round',
       'timeformat'],
      dtype='object')

### Differential Calculation
A differential is the difference in the amount of times a fighter executes a technique vs the amount of times his opponent does.
To calculate this, we transform our original table so that each row has a fighter AND his opponent.

In [20]:
dif_calc_df = functions.merge_fighter_instances(rounds, rounds=False)
dif_calc_df.head()

,fighter_0,kd_0,sba_0,ps_0,rev_0,ss_s_0,ss_a_0,h_ss_s_0,h_ss_a_0,b_ss_s_0,...,td_s_1,td_a_1,outcome_1,fighter_id_1,round_1,date_1,time_1,final_round_1,timeformat_1,inst_id_1
0,Robert Whittaker,0,0,0,0,11,30,4,22,1,...,0,0,L,9ce6d5a03af801b7,1,"July 25, 2020",5:00,5,5 Rnd (5-5-5-5-5),11f715fa5e825e519ce6d5a03af801b7
1,Robert Whittaker,0,0,0,0,11,30,4,22,1,...,0,0,L,9ce6d5a03af801b7,2,"July 25, 2020",5:00,5,5 Rnd (5-5-5-5-5),11f715fa5e825e519ce6d5a03af801b7
2,Robert Whittaker,0,0,0,0,11,30,4,22,1,...,0,0,L,9ce6d5a03af801b7,3,"July 25, 2020",5:00,5,5 Rnd (5-5-5-5-5),11f715fa5e825e519ce6d5a03af801b7
3,Robert Whittaker,0,0,0,0,11,30,4,22,1,...,0,0,L,9ce6d5a03af801b7,4,"July 25, 2020",5:00,5,5 Rnd (5-5-5-5-5),11f715fa5e825e519ce6d5a03af801b7
4,Robert Whittaker,0,0,0,0,11,30,4,22,1,...,0,0,L,9ce6d5a03af801b7,5,"July 25, 2020",5:00,5,5 Rnd (5-5-5-5-5),11f715fa5e825e519ce6d5a03af801b7


##### In this dataframe, each row has two fighters. The first is suffixed with a _0 and the second with a _1.
##### In order to get the significant strike attempts per round differential (ss_a_pr_di), all we need to do 
##### is subtract ss_a_1 from ss_a_0

In [21]:
dif_calc_df['ss_a_pr_di'] = dif_calc_df['ss_a_0'] - dif_calc_df['ss_a_1']

In [33]:
dif_calc_df.loc[len(dif_calc_df)-5:,['ss_a_pr_di', 'ss_a_0', 'ss_a_1']]

,ss_a_pr_di,ss_a_0,ss_a_1
35998,-14,3,17
35999,-7,1,8
36000,-3,0,3
36001,1,28,27
36002,-4,1,5


##### In the list above, the first fighter only attempted 3 strikes, while his opponent attempted 17, resulting in a differential of -14

#### Calculating for all stats
##### I wrote a function that calculates the differentials for
##### per round counts, per 1 minute rates, and per 15 minute rates

In [34]:
advanced_stats = functions.calculate_stats_alt(rounds, 
                                               ['kd', 'sba', 'ps', 'rev', 
                                                'ss_s', 'ss_a', 'h_ss_s', 
                                                'h_ss_a','b_ss_s', 'b_ss_a', 
                                                'l_ss_s', 'l_ss_a', 'd_ss_s', 
                                                'd_ss_a', 'c_ss_s', 'c_ss_a', 
                                                'g_ss_s', 'g_ss_a', 'ts_s', 
                                                'ts_a', 'td_s', 'td_a'])

calculating minutes



/home/max/sports_science/match_finder/src/functions.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round_id'] = df['bout_id']+df['round']


combining rows

calculating differentials for kd

calculating differentials for sba

calculating differentials for ps

calculating differentials for rev

calculating differentials for ss_s

calculating differentials for ss_a

calculating differentials for h_ss_s

calculating differentials for h_ss_a

calculating differentials for b_ss_s

calculating differentials for b_ss_a

calculating differentials for l_ss_s

calculating differentials for l_ss_a

calculating differentials for d_ss_s

calculating differentials for d_ss_a

calculating differentials for c_ss_s

calculating differentials for c_ss_a

calculating differentials for g_ss_s

calculating differentials for g_ss_a

calculating differentials for ts_s

calculating differentials for ts_a

calculating differentials for td_s

calculating differentials for td_a

cleaning df



In [37]:
advanced_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25772 entries, 0 to 12885
Columns: 144 entries, fighter to td_a_p15m_di
dtypes: float64(89), int64(45), object(9), timedelta64[ns](1)
memory usage: 28.5+ MB


Because this dataframe has 144 rows, I'm going to split it up into the following advanced stats tables:
- Grappling, includes
    - Takedowns
    - Submission Attempts
    - Reversals
    - Guard Passes
- Strikes
    - Significant Strikes
    - Total Strikes
    - Knockdowns
- Strikes by Target
    - Head
    - Leg
    - Body
- Strikes by Position
    - Distance
    - Ground 
    - Clinch

### Functions

In [53]:
def get_metric_columns(stat, metrics_list=None):
    """
    input: stat (string) - the abbreviation of a stat from the dataframe
    output: list of column names for:
            - per 1 minute rate (_p1m)
            - per 15 minute rate (_p15m)
            - per round differential (_pr_di)
            - per 1 minute differential (_p1m_di)
            - per 15 minute differential (_p15m_di)
    """
    if metrics_list:
        metrics = metrics_list
    else:
        metrics = ['_p1m', '_p15m', '_pr_di', '_p1m_di', '_p15m_di']
    
    columns = [stat]
    for metric in metrics:
        # combine the stat and the metric name and add to columns list
        columns.append(stat+metric)
        
    return columns

In [57]:
def get_advanced_stats_table_columns(table_stats, metrics_list=None):
    """
    input: table_stats(list) - list of stats that will go in the table
    output: list of column names for the advanced stats table
    """
    columns = ['bout_id', 'fighter_id', 'round', 'fighter']
    for stat in table_stats:
        columns = columns + get_metric_columns(stat, metrics_list=metrics_list)
    
    return columns

#### Grappling

In [58]:
get_advanced_stats_table_columns(['td_s', 'td_a', 'sba', 'rev', 'ps'])

['bout_id',
 'fighter_id',
 'round',
 'fighter',
 'td_s',
 'td_s_p1m',
 'td_s_p15m',
 'td_s_pr_di',
 'td_s_p1m_di',
 'td_s_p15m_di',
 'td_a',
 'td_a_p1m',
 'td_a_p15m',
 'td_a_pr_di',
 'td_a_p1m_di',
 'td_a_p15m_di',
 'sba',
 'sba_p1m',
 'sba_p15m',
 'sba_pr_di',
 'sba_p1m_di',
 'sba_p15m_di',
 'rev',
 'rev_p1m',
 'rev_p15m',
 'rev_pr_di',
 'rev_p1m_di',
 'rev_p15m_di',
 'ps',
 'ps_p1m',
 'ps_p15m',
 'ps_pr_di',
 'ps_p1m_di',
 'ps_p15m_di']

In [62]:
grappling_stats = ['td_s', 'td_a', 'sba', 'rev', 'ps']
grappling_columns = get_advanced_stats_table_columns(grappling_stats)

grappling = advanced_stats.loc[:,grappling_columns]
grappling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25772 entries, 0 to 12885
Data columns (total 34 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   bout_id       25772 non-null  object 
 1   fighter_id    25772 non-null  object 
 2   round         25772 non-null  object 
 3   fighter       25772 non-null  object 
 4   td_s          25772 non-null  int64  
 5   td_s_p1m      25772 non-null  float64
 6   td_s_p15m     25772 non-null  float64
 7   td_s_pr_di    25772 non-null  int64  
 8   td_s_p1m_di   25772 non-null  float64
 9   td_s_p15m_di  25772 non-null  float64
 10  td_a          25772 non-null  int64  
 11  td_a_p1m      25772 non-null  float64
 12  td_a_p15m     25772 non-null  float64
 13  td_a_pr_di    25772 non-null  int64  
 14  td_a_p1m_di   25772 non-null  float64
 15  td_a_p15m_di  25772 non-null  float64
 16  sba           25772 non-null  int64  
 17  sba_p1m       25772 non-null  float64
 18  sba_p15m      25772 non-nu

##### Send to SQL

In [ ]:
grappling.to_csv('../../data/ufcstats_data/advanced_stats/grappling_adv.csv')
grappling.to_sql('grappling_adv')

#### Strikes

In [41]:
list(advanced_stats.columns)

['fighter',
 'kd',
 'sba',
 'ps',
 'rev',
 'ss_s',
 'ss_a',
 'h_ss_s',
 'h_ss_a',
 'b_ss_s',
 'b_ss_a',
 'l_ss_s',
 'l_ss_a',
 'd_ss_s',
 'd_ss_a',
 'c_ss_s',
 'c_ss_a',
 'g_ss_s',
 'g_ss_a',
 'ts_s',
 'ts_a',
 'td_s',
 'td_a',
 'outcome',
 'bout_id',
 'fighter_id',
 'round',
 'date',
 'time',
 'final_round',
 'timeformat',
 'inst_id',
 'round_length',
 'minutes',
 'kd_p1m',
 'kd_p15m',
 'sba_p1m',
 'sba_p15m',
 'ps_p1m',
 'ps_p15m',
 'rev_p1m',
 'rev_p15m',
 'ss_s_p1m',
 'ss_s_p15m',
 'ss_a_p1m',
 'ss_a_p15m',
 'h_ss_s_p1m',
 'h_ss_s_p15m',
 'h_ss_a_p1m',
 'h_ss_a_p15m',
 'b_ss_s_p1m',
 'b_ss_s_p15m',
 'b_ss_a_p1m',
 'b_ss_a_p15m',
 'l_ss_s_p1m',
 'l_ss_s_p15m',
 'l_ss_a_p1m',
 'l_ss_a_p15m',
 'd_ss_s_p1m',
 'd_ss_s_p15m',
 'd_ss_a_p1m',
 'd_ss_a_p15m',
 'c_ss_s_p1m',
 'c_ss_s_p15m',
 'c_ss_a_p1m',
 'c_ss_a_p15m',
 'g_ss_s_p1m',
 'g_ss_s_p15m',
 'g_ss_a_p1m',
 'g_ss_a_p15m',
 'ts_s_p1m',
 'ts_s_p15m',
 'ts_a_p1m',
 'ts_a_p15m',
 'td_s_p1m',
 'td_s_p15m',
 'td_a_p1m',
 'td_a_p15m',
 'k

In [ ]:
zzz

In [4]:
drop_columns = ['fighter', 'kd', 'sub_att', 'pass', 'rev',
       'outcome', 'total_str_s', 'total_str_a', 'td_s', 'td_a',
       'date', 'time', 'final_round', 'timeformat', 'date', 'fighter_id',
       'bout_id', 'round_length', 'minutes', 'inst_id']

In [105]:
takedown_stats.drop(drop_columns, axis=1, inplace=True)

In [106]:
takedown_stats.to_csv('../../data/takedown_stats.csv')

In [106]:
takedown_stats.to_csv('../../data/takedown_stats.csv')

In [2]:
from src import functions
total_strike_stats = functions.calculate_stats(general_data, ['total_str'])

formatting data

calculating minutes



/home/max/sports_science/match_finder/src/functions.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round_id'] = df['bout_id']+df['round']


calculating accuracy for total_str

combining rows

calculating defense for total_str

calculating differentials for total_str

cleaning df



In [5]:
total_strike_stats.drop(drop_columns, axis=1, inplace=True)

In [6]:
total_strike_stats.to_csv('../../total_strike_stats.csv', index=False)

In [81]:
grappling_stats = functions.calculate_stats_alt(general_data, ['rev', 'pass', 'sub_att'])

formatting data

calculating minutes



/home/max/sports_science/match_finder/src/functions.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round_id'] = df['bout_id']+df['round']


combining rows

calculating differentials for rev

calculating differentials for pass

calculating differentials for sub_att

cleaning df



In [107]:
grappling_stats.drop(drop_columns, axis=1, inplace=True)

In [108]:
grappling_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25772 entries, 0 to 12885
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   round            25772 non-null  object 
 1   bout_link        25772 non-null  object 
 2   fighter_link     25772 non-null  object 
 3   rev_p1m          25772 non-null  float64
 4   rev_p15m         25772 non-null  float64
 5   pass_p1m         25772 non-null  float64
 6   pass_p15m        25772 non-null  float64
 7   sub_att_p1m      25772 non-null  float64
 8   sub_att_p15m     25772 non-null  float64
 9   rev_pr_di        25772 non-null  int64  
 10  rev_p1m_di       25772 non-null  float64
 11  rev_p15m_di      25772 non-null  float64
 12  pass_pr_di       25772 non-null  int64  
 13  pass_p1m_di      25772 non-null  float64
 14  pass_p15m_di     25772 non-null  float64
 15  sub_att_pr_di    25772 non-null  int64  
 16  sub_att_p1m_di   25772 non-null  float64
 17  sub_att_p15m

In [109]:
grappling_stats.to_csv('../../data/grappling_stats.csv', index=False)

In [7]:
knockdown_stats = functions.calculate_stats_alt(general_data, ['kd'])

formatting data

calculating minutes



/home/max/sports_science/match_finder/src/functions.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round_id'] = df['bout_id']+df['round']


combining rows

calculating differentials for kd

cleaning df



In [8]:
knockdown_stats.drop(drop_columns, axis=1, inplace=True)

In [9]:
knockdown_stats

,round,bout_link,fighter_link,kd_p1m,kd_p15m,kd_pr_di,kd_p1m_di,kd_p15m_di
0,1,http://www.ufcstats.com/fight-details/11f715fa...,http://www.ufcstats.com/fighter-details/e1147d...,0.000000,0.000000,0,0.000000,0.000000
1,2,http://www.ufcstats.com/fight-details/11f715fa...,http://www.ufcstats.com/fighter-details/e1147d...,0.200000,3.000000,1,0.200000,3.000000
2,3,http://www.ufcstats.com/fight-details/11f715fa...,http://www.ufcstats.com/fighter-details/e1147d...,0.000000,0.000000,0,0.000000,0.000000
3,4,http://www.ufcstats.com/fight-details/11f715fa...,http://www.ufcstats.com/fighter-details/e1147d...,0.000000,0.000000,0,0.000000,0.000000
4,5,http://www.ufcstats.com/fight-details/11f715fa...,http://www.ufcstats.com/fighter-details/e1147d...,0.000000,0.000000,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...
12881,1,http://www.ufcstats.com/fight-details/693e4a0b...,http://www.ufcstats.com/fighter-details/669a3c...,0.000000,0.000000,0,0.000000,0.000000
12882,2,http://www.ufcstats.com/fight-details/693e4a0b...,http://www.ufcstats.com/fighter-details/669a3c...,0.000000,0.000000,-1,-0.759494,-11.392405
12883,1,http://www.ufcstats.com/fight-details/c6f85419...,http://www.ufcstats.com/fighter-details/a54a35...,0.350877,5.263158,1,0.350877,5.263158
12884,1,http://www.ufcstats.com/fight-details/1db8bed8...,http://www.ufcstats.com/fighter-details/911fb2...,0.000000,0.000000,0,0.000000,0.000000


In [10]:
knockdown_stats.to_csv('../../knockdown_stats.csv', index=False)

In [76]:
query = """
SELECT strikes_cleaned.*, "Date", "Time", 
"Round" as final_round, "Timeformat"
FROM strikes_cleaned
JOIN bouts ON bouts.link = strikes_cleaned.bout_link
JOIN events ON events.link = bouts.event_link
"""

strike_data = pd.read_sql(query, engine)

In [77]:
strike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26452 entries, 0 to 26451
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   fighter       26452 non-null  object
 1   round         26452 non-null  int64 
 2   bout_link     26452 non-null  object
 3   outcome       26452 non-null  object
 4   fighter_link  26452 non-null  object
 5   sig_str_s     26452 non-null  int64 
 6   sig_str_a     26452 non-null  int64 
 7   head_s        26452 non-null  int64 
 8   head_a        26452 non-null  int64 
 9   body_s        26452 non-null  int64 
 10  body_a        26452 non-null  int64 
 11  leg_s         26452 non-null  int64 
 12  leg_a         26452 non-null  int64 
 13  distance_s    26452 non-null  int64 
 14  distance_a    26452 non-null  int64 
 15  clinch_s      26452 non-null  int64 
 16  clinch_a      26452 non-null  int64 
 17  ground_s      26452 non-null  int64 
 18  ground_a      26452 non-null  int64 
 19  Date

In [78]:
sig_str_stats = functions.calculate_stats(strike_data, ['sig_str'])

formatting data

calculating minutes



/home/max/sports_science/match_finder/src/functions.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round_id'] = df['bout_id']+df['round']


calculating accuracy for sig_str

combining rows

calculating defense for sig_str

calculating differentials for sig_str

cleaning df



In [121]:
drop_columns = ['fighter', 'outcome', 'sig_str_s',
       'sig_str_a', 'head_s', 'head_a', 'body_s', 'body_a', 'leg_s', 'leg_a',
       'distance_s', 'distance_a', 'clinch_s', 'clinch_a', 'ground_s',
       'ground_a', 'date', 'time', 'final_round', 'timeformat', 'date',
       'fighter_id', 'bout_id', 'round_length', 'minutes', 'inst_id']

In [123]:
sig_str_stats.drop(drop_columns, axis=1, inplace=True)

In [125]:
sig_str_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25772 entries, 0 to 12885
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   round              25772 non-null  object 
 1   bout_link          25772 non-null  object 
 2   fighter_link       25772 non-null  object 
 3   sig_str_a_p1m      25772 non-null  float64
 4   sig_str_s_p1m      25772 non-null  float64
 5   sig_str_a_p15m     25772 non-null  float64
 6   sig_str_s_p15m     25772 non-null  float64
 7   sig_str_ac         25646 non-null  object 
 8   sig_str_de         25646 non-null  object 
 9   sig_str_s_pr_di    25772 non-null  int64  
 10  sig_str_a_pr_di    25772 non-null  int64  
 11  sig_str_s_p1m_di   25772 non-null  float64
 12  sig_str_a_p1m_di   25772 non-null  float64
 13  sig_str_s_p15m_di  25772 non-null  float64
 14  sig_str_a_p15m_di  25772 non-null  float64
dtypes: float64(8), int64(2), object(5)
memory usage: 3.1+ MB


In [126]:
sig_str_stats.to_csv('../../data/sig_str_stats.csv', index=False)

In [79]:
body_ss_stats = functions.calculate_stats(strike_data, ['body'])
head_ss_stats = functions.calculate_stats(strike_data, ['head'])
leg_ss_stats = functions.calculate_stats(strike_data, ['leg'])
distance_ss_stats = functions.calculate_stats(strike_data, ['distance'])
clinch_ss_stats = functions.calculate_stats(strike_data, ['clinch'])
ground_ss_stats = functions.calculate_stats(strike_data, ['ground'])

formatting data

calculating minutes

calculating accuracy for body

combining rows

calculating defense for body

calculating differentials for body

cleaning df

formatting data

calculating minutes

calculating accuracy for head

combining rows

calculating defense for head

calculating differentials for head

cleaning df

formatting data

calculating minutes

calculating accuracy for leg

combining rows

calculating defense for leg

calculating differentials for leg

cleaning df

formatting data

calculating minutes

calculating accuracy for distance

combining rows

calculating defense for distance

calculating differentials for distance

cleaning df

formatting data

calculating minutes

calculating accuracy for clinch

combining rows

calculating defense for clinch

calculating differentials for clinch

cleaning df

formatting data

calculating minutes

calculating accuracy for ground

combining rows

calculating defense for ground

calculating differentials for ground

cleaning

In [127]:
body_ss_stats.drop(drop_columns, axis=1, inplace=True)
head_ss_stats.drop(drop_columns, axis=1, inplace=True)
leg_ss_stats.drop(drop_columns, axis=1, inplace=True)
distance_ss_stats.drop(drop_columns, axis=1, inplace=True)
clinch_ss_stats.drop(drop_columns, axis=1, inplace=True)
ground_ss_stats.drop(drop_columns, axis=1, inplace=True)

In [128]:
body_ss_stats.to_csv('../../data/body_ss_stats.csv', index=False)
head_ss_stats.to_csv('../../data/head_ss_stats.csv', index=False)
leg_ss_stats.to_csv('../../data/leg_ss_stats.csv', index=False)
distance_ss_stats.to_csv('../../data/distance_ss_stats.csv', index=False)
clinch_ss_stats.to_csv('../../data/clinch_ss_stats.csv', index=False)
ground_ss_stats.to_csv('../../data/ground_ss_stats.csv', index=False)

## Last minute addition, adding to sql

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from sqlalchemy import create_engine
from src import local


# Credentials
USER = local.user 
PASS = local.password
HOST = local.host
PORT = local.port

#create engine
engine = create_engine(f'postgresql://{USER}:{PASS}@{HOST}:{PORT}/match_finder')